# Limpieza de Datos

### Resultados electorales

In [1]:
import pandas as pd
import numpy as np
import re

Cargamos los datos desde los CSV descargados.

In [148]:
file16 = "..\\..\\raw data\\rawmesasgip2016.csv"

In [149]:
data16 = pd.read_csv(file16,header= None)

Aplicamos una función de limpieza de datos, dado que estos vienen en un inconveniente formato en el que todos los valores están en la misma casilla, sin separadores, y sólo sabemos qué significan por su posición en el string.

In [150]:
def limpieza(dataframe):
    '''Esta función limpiará el archivo original de resultados de las elecciones. 
    Al tratarse de un archivo con una sola columna que "esconde" en diversas posiciones la información,
    habrá que realizar numerosas operaciones para obtener el Dataframe deseado'''
    #los siguientes slicing sacan datos del csv con una única columna, fuente inicial de data.
    dataframe['cod.municipio'] = dataframe[0].astype(str).str[13:16]
    dataframe['cod.distrito'] = dataframe[0].astype(str).str[16:18]
    dataframe['cod.sección'] = dataframe[0].astype(str).str[18:23]
    dataframe['censo'] = dataframe[0].astype(str).str[26:30]
    dataframe['votosa'] = dataframe[0].astype(str).str[65:72]
    dataframe['votosb'] = dataframe[0].astype(str).str[73:79]
    dataframe['votosc'] = dataframe[0].astype(str).str[79:86]
    #suma de las columnas de votos, para obtener el voto total (válidos, nulos y blancos)
    dataframe[['votosa', 'votosb','votosc']]= dataframe[['votosa', 'votosb','votosc']].apply(pd.to_numeric)
    dataframe['votos_total']= (dataframe['votosa'])+(dataframe['votosb'])+(dataframe['votosc'])
    #pasamos "censo" a numérico para poder agrupar los datos de las secciones que tienen mesa A y B
    dataframe['censo']= dataframe['censo'].apply(pd.to_numeric)
    dataframe.drop(columns=[0,'votosa','votosb','votosc'], inplace=True)
    # Eliminamos el string "A", "B", "U" de las mesas/códigos de sección.
    # agrupamos según igual cod.municipio, distrito y sección, sumando los censos y votos totales de las mesas A y B donde las hay.
    dataframe['cod.sección'] = dataframe['cod.sección'].str.extract('(\d+)')
    dataframe = dataframe.groupby(['cod.municipio', 'cod.distrito', 'cod.sección']).agg({
        'censo': 'sum',
        'votos_total': 'sum'
    }).reset_index()
    #Creamos la columna con el código de la sección, y eliminamos las columnas de sus componentes.
    #Esta será la columna que servirá para su posterior unión a los datos de renta
    dataframe['Sección']= dataframe['cod.municipio']+dataframe['cod.distrito']+dataframe['cod.sección']
    dataframe.drop(['cod.municipio','cod.distrito','cod.sección'], inplace=True, axis=1)
    return dataframe

In [151]:
data16 = limpieza(data16)

Pasamos por el mismo procedimiento de limpieza los datos de 2019

In [143]:
file19 = "..\\..\\raw data\\rawmesasgip2019.csv"

In [144]:
data19 = pd.read_csv(file19,header= None)

In [145]:
limpieza(data19)

,censo,votos_total,Sección
0,253,163,00101001
1,359,234,00201001
2,549,360,00301001
3,230,133,00401001
4,1274,822,00501001
...,...,...,...
540,87,56,90401001
541,160,123,90501001
542,128,87,90601001
543,118,82,90701001


En lo respectivo a la renta, esta viene en un dataset más manejable. Aún así, habrá que detectar y tratar valores NaN, cambiar types para su mejor manejo, y substraer los datos concretos en nuevas columnas para su posterior trabajo.

In [38]:
file_renta16 = "..\\..\\raw data\\RentaGipu2016secciones.csv"

In [65]:
data_renta16 = pd.read_csv(file_renta16,encoding='latin-1',header= 0, sep=";")

In [66]:
data_renta16.rename(columns={'Secciones':'Sección'}, inplace=True)

In [41]:
#Detectamos valores NaN de renta, para cambiar sin entrar en conflictos de tipo a Int
data_renta16[data_renta16['Total'].isna()]

,Municipios,Distritos,Sección,Indicadores de renta media y mediana,Periodo,Total
346,20069 Donostia/San Sebastián,2006903 Donostia/San Sebastián distrito 03,2006903033 Donostia/San Sebastián sección 03033,Renta neta media por persona,2016,NaN
347,20069 Donostia/San Sebastián,2006903 Donostia/San Sebastián distrito 03,2006903034 Donostia/San Sebastián sección 03034,Renta neta media por persona,2016,NaN


In [67]:
#los rellenamos con la media. Luego se omitirán en el análisis pordetallado
data_renta16['Total'].fillna(value=data_renta16['Total'].mean(), inplace=True) 

In [68]:
data_renta16['Total']=data_renta16['Total']*1000

In [73]:
#Funciones para sacar de la casilla "Sección" su dato numérico por un lado y su nombre por otro.
def ultimas(texto):
    palabras = texto.split()
    return ' '.join(palabras[1:])
def eliminar_ultimas(texto):
    palabras = texto.split()
    return (palabras[0])

data_renta16['Nombre Sección'] = data_renta16['Sección'].apply(ultimas)
data_renta16['Sección']= data_renta16['Sección'].apply(eliminar_ultimas)
data_renta16['Sección'] = data_renta16['Sección'].str.slice(2)

In [21]:
data_renta16['Total'] = data_renta16['Total'].astype(str)


In [71]:
data_renta16['Total'] = data_renta16['Total'].astype(float).astype(int)


In [72]:
data_renta16

,Municipios,Distritos,Sección,Indicadores de renta media y mediana,Periodo,Total
0,20001 Abaltzisketa,2000101 Abaltzisketa distrito 01,2000101001 Abaltzisketa sección 01001,Renta neta media por persona,2016,14209
1,20002 Aduna,2000201 Aduna distrito 01,2000201001 Aduna sección 01001,Renta neta media por persona,2016,15485
2,20003 Aizarnazabal,2000301 Aizarnazabal distrito 01,2000301001 Aizarnazabal sección 01001,Renta neta media por persona,2016,12996
3,20004 Albiztur,2000401 Albiztur distrito 01,2000401001 Albiztur sección 01001,Renta neta media por persona,2016,16226
4,20005 Alegia,2000501 Alegia distrito 01,2000501001 Alegia sección 01001,Renta neta media por persona,2016,12678
...,...,...,...,...,...,...
540,20903 Astigarraga,2090301 Astigarraga distrito 01,2090301003 Astigarraga sección 01003,Renta neta media por persona,2016,13523
541,20904 Baliarrain,2090401 Baliarrain distrito 01,2090401001 Baliarrain sección 01001,Renta neta media por persona,2016,13470
542,20905 Orendain,2090501 Orendain distrito 01,2090501001 Orendain sección 01001,Renta neta media por persona,2016,14906
543,20906 Altzaga,2090601 Altzaga distrito 01,2090601001 Altzaga sección 01001,Renta neta media por persona,2016,13192


In [75]:
#Los datos que nos interesan son el código de la sección, su nombre, y el dato de renta.
data_renta16_final = data_renta16.iloc[:, [6,2,5]]

In [76]:
data_renta16_final

,Nombre Sección,Sección,Total
0,Abaltzisketa sección 01001,00101001,14209
1,Aduna sección 01001,00201001,15485
2,Aizarnazabal sección 01001,00301001,12996
3,Albiztur sección 01001,00401001,16226
4,Alegia sección 01001,00501001,12678
...,...,...,...
540,Astigarraga sección 01003,90301003,13523
541,Baliarrain sección 01001,90401001,13470
542,Orendain sección 01001,90501001,14906
543,Altzaga sección 01001,90601001,13192


In [107]:
file_renta19 = "..\\..\\raw data\\RentaGipu2019secciones.csv"

In [108]:
data_renta19 = pd.read_csv(file_renta19,encoding='latin-1',header= 0, sep=";")

In [109]:
data_renta19.rename(columns={'Secciones':'Sección'}, inplace=True)

In [110]:
#Detectamos valores NaN de renta, para cambiar sin entrar en conflictos de tipo a Int
data_renta19[data_renta19['Total'].isna()]

,Municipios,Distritos,Sección,Indicadores de renta media y mediana,Periodo,Total


En este caso, no habrá que suplantar valores por la media.

In [111]:
data_renta19['Total']=data_renta19['Total']*1000

In [112]:
#Funciones para sacar de la casilla "Sección" su dato numérico por un lado y su nombre por otro.
def ultimas(texto):
    palabras = texto.split()
    return ' '.join(palabras[1:])
def eliminar_ultimas(texto):
    palabras = texto.split()
    return (palabras[0])

data_renta19['Nombre Sección'] = data_renta19['Sección'].apply(ultimas)
data_renta19['Sección']= data_renta19['Sección'].apply(eliminar_ultimas)
data_renta19['Sección'] = data_renta19['Sección'].str.slice(2)

In [113]:
data_renta19['Total'] = data_renta19['Total'].astype(str)

In [114]:
data_renta19['Total'] = data_renta19['Total'].astype(float).astype(int)


In [116]:
#Los datos que nos interesan son el código de la sección, su nombre, y el dato de renta.
data_renta19_final = data_renta19.iloc[:, [6,2,5]]

In [117]:
data_renta19_final

,Nombre Sección,Sección,Total
0,Abaltzisketa sección 01001,00101001,17166
1,Aduna sección 01001,00201001,16825
2,Aizarnazabal sección 01001,00301001,14763
3,Albiztur sección 01001,00401001,16659
4,Alegia sección 01001,00501001,14319
...,...,...,...
540,Astigarraga sección 01003,90301003,15474
541,Baliarrain sección 01001,90401001,15116
542,Orendain sección 01001,90501001,16087
543,Altzaga sección 01001,90601001,14018


In [147]:
data16

,cod.municipio,cod.distrito,cod.sección,censo,votos_total
0,001,01,001,246,133
1,002,01,001,346,203
2,003,01,001,547,308
3,004,01,001,252,121
4,005,01,001,598,362
...,...,...,...,...,...
835,904,01,001,95,44
836,905,01,001,145,100
837,906,01,001,119,87
838,907,01,001,118,85


In [152]:
data_renta16_final.merge(data16, on='Sección', how='outer')

,Nombre Sección,Sección,Total,censo,votos_total
0,Abaltzisketa sección 01001,00101001,14209000.0,246.0,133.0
1,Aduna sección 01001,00201001,15485000.0,346.0,203.0
2,Aizarnazabal sección 01001,00301001,12996000.0,547.0,308.0
3,Albiztur sección 01001,00401001,16226000.0,252.0,121.0
4,Alegia sección 01001,00501001,12678000.0,1260.0,738.0
...,...,...,...,...,...
541,Baliarrain sección 01001,90401001,13470000.0,95.0,44.0
542,Orendain sección 01001,90501001,14906000.0,145.0,100.0
543,Altzaga sección 01001,90601001,13192000.0,119.0,87.0
544,Gaztelu sección 01001,90701001,13014000.0,118.0,85.0
